# Fixing triangular meshes with trimesh

Consider: 
* The function `txttoarray` will only work for txt files inside the folder this notebook is in.
* Mesh will be made watertight by filling holes, deleting repeted vertices and faces, and by merging vertices that are close together. This distance can be toggled through the variable `merge_tolerance`.

In [1]:
%reset -f
import time
import numpy as np
import trimesh


def txttoarray(filename):
    file = open(filename, "r").read().split("\n")
    datos=[]
    for i in range(len(file)):
        datos.append(file[i].split())
    return(datos)

######################### #########################
######################### #########################

toc = time.time()
    
# Replace "surf_d02_stern.face" and "vert" with filename (txt extention)
face=txttoarray("Meshing_1a0j/1a0j_d08.face")
face=[i for i in face if i!=[]]

vert=txttoarray("Meshing_1a0j/1a0j_d08.vert")
vert=[i for i in vert if i!=[]]

vert_txt=np.float_(vert) ; vert_norm=vert_txt[:,3:6] ; vert=vert_txt[:,:3]
face=np.int_(face); face= face[:,:3]-1

######################### #########################
######################### #########################
    
mesh = trimesh.Trimesh(vertices=vert,faces=face,vertex_normals=vert_norm,validate=True,process=True)
mesh.fill_holes()
mesh.process()

while not mesh.is_watertight:
    merge_tolerance = 0.005
    needy_faces     = trimesh.repair.broken_faces(mesh)
    for vert_nf in mesh.faces[needy_faces]:
        for nf in vert_nf:
            for c,check in enumerate(np.linalg.norm(mesh.vertices[vert_nf]-mesh.vertices[nf],axis=1)):
                if (check<merge_tolerance) & (0<check):
                    mesh.vertices[nf]=mesh.vertices[vert_nf[c]]
                    
mesh.fill_holes()
mesh.process()
            
                
tic = time.time()
print("Total time:",round(tic-toc,3),"[s]")

Total time: 0.649 [s]


In [2]:
print(mesh)

<trimesh.Trimesh(vertices.shape=(34278, 3), faces.shape=(68548, 3))>


In [3]:
mesh_split = mesh.split()
print(mesh)
print(mesh_split)
print("len de mesh_split: ",len(mesh_split))

<trimesh.Trimesh(vertices.shape=(34278, 3), faces.shape=(68548, 3))>
[<trimesh.Trimesh(vertices.shape=(33286, 3), faces.shape=(66580, 3))>
 <trimesh.Trimesh(vertices.shape=(394, 3), faces.shape=(784, 3))>
 <trimesh.Trimesh(vertices.shape=(290, 3), faces.shape=(576, 3))>
 <trimesh.Trimesh(vertices.shape=(198, 3), faces.shape=(392, 3))>
 <trimesh.Trimesh(vertices.shape=(110, 3), faces.shape=(216, 3))>]
len de mesh_split:  5


In [4]:
mesh_ses = mesh_split[0]
print(mesh_ses)

<trimesh.Trimesh(vertices.shape=(33286, 3), faces.shape=(66580, 3))>


In [5]:
mesh_ses.show()

In [6]:
mesh_ses_faces = mesh_ses.faces + 1
mesh_ses_vert = mesh_ses.vertices
print(mesh_ses_faces, mesh_ses_vert)

[[  697   716   423]
 [29015 10705 20557]
 [ 4920 29015 22091]
 ..., 
 [21480 18662 22874]
 [ 9653  7274 22876]
 [11797 12411 23157]] [[ -0.     42.92   54.444]
 [ -0.     50.892  60.642]
 [ 20.48   30.807  34.435]
 ..., 
 [ 20.963  55.039  73.203]
 [ 31.203  28.898  47.504]
 [ 31.203  41.315  42.017]]


In [7]:
np.savetxt("Meshing_1a0j/1a0j_d08_split.face",mesh_ses_faces,fmt="%i")
np.savetxt("Meshing_1a0j/1a0j_d08_split.vert",mesh_ses_vert)

In [ ]:
#mesh: face - vert
#d01: 7384 - 3690
#d02: 15684 - 7842
#d04: 32524 - 16260
#d08: 66580 - 33286
#d16: 135644 - 67816